In [1]:
from simulate_game import simulate_game
from competitive_sudoku.sudoku import load_sudoku

import importlib
import sys
import io
import os
import pandas as pd
import matplotlib.pyplot as plt
import itertools

from tqdm import tqdm

In [2]:
# Winner 0 if it was a draw, 1 if player 1 won, 2 if player 2 won
# Reason:
# "regular" if it was played till the end
# "taboo" if a taboo move was played
# "invalid" if invalid move was played
# "illegal" if illegal move was played
# "no move" if no move was supplied


def run_simulation(first_player: str, second_player: str, board_name: str, time: float) -> set:
    player1 = importlib.import_module(first_player + '.sudokuai').SudokuAI()
    player2 = importlib.import_module(second_player + '.sudokuai').SudokuAI()
    player1.player_number = 1
    player2.player_number = 2
    
    # We do not check for specific ai, just give it to all of them
    player1.solve_sudoku_path = 'bin\\solve_sudoku.exe'
    player2.solve_sudoku_path = 'bin\\solve_sudoku.exe'
    
    board = load_sudoku(board_name)
    
    # Note for future, we do not clean up files
    
    old_stdout = sys.stdout
    sys.stdout = buffer = io.StringIO()
    
    simulate_game(board, player1, player2, solve_sudoku_path='bin\\solve_sudoku.exe', calculation_time=time)
    
    sys.stdout = old_stdout
    result = buffer.getvalue()
    
    last_line = result.splitlines()[-1]
    splitted_line = last_line.split(".")
    
    if len(splitted_line) == 2:
        winner = {"The game ends in a draw": 0, "Player 1 wins the game":1, 
                  "Player 2 wins the game":2}[splitted_line[0]]
        reason = "regular"
    
    else:
        winner = {" The game ends in a draw": 0, " Player 1 wins the game":1, 
                  " Player 2 wins the game":2}[splitted_line[1]]
        reason = {"taboo move": "taboo", "valid move": "invalid", "legal move": "illegal", 
                  "s supplied": "no move"}[splitted_line[0][-10:]]
    
    return winner, reason

In [3]:
from distutils.dir_util import copy_tree

def create_new_file(C, t, s):

    copy_tree("MC_tests//team41_A3_MC_2_False_max", f"MC_tests//team41_A3_MC_{C}_{t}_{s}")
    os.remove(f"MC_tests//team41_A3_MC_{C}_{t}_{s}//sudokuai.py")

    with open("MC_tests//team41_A3_MC_2_False_max//sudokuai.py") as old_file:
        with open(f"MC_tests//team41_A3_MC_{C}_{t}_{s}//sudokuai.py", "w") as new_file:
            for line in old_file:
                if line == "global_C = 2\n":
                    new_file.write(f"global_C = {C}\n")

                elif line == "global_total = False\n":
                    new_file.write(f"global_total = {t}\n")

                elif line == "global_selection = \"max\"\n":
                    new_file.write(f"global_selection = \"{s}\"\n")

                else:
                    new_file.write(line)

In [4]:
Cs = [0, 2, 5, 10]
totals = [False, True]
selections = ["max", "robust"]

In [ ]:
boards = os.listdir("boards")
df = pd.DataFrame(columns = ["board", "starting", "C", "totals", "selection" "result", "reason"])

for setup in tqdm(list(itertools.product(Cs, totals, selections))):
    if setup != (2,False,"max"):
        create_new_file(setup[0], setup[1], setup[2])
        
        for board in boards:
            print(board)
            winner, reason = run_simulation(f"MC_tests.team41_A3_MC_{setup[0]}_{setup[1]}_{setup[2]}","random_player", f'boards//{board}', 0.5)
            result = {0: 0, 1:1, 2:-1}[winner]
            new_row = pd.DataFrame({"board": board[:-4], "starting": True, "C": setup[0], "totals": setup[1],
                                    "selection": setup[2], "result": result, "reason": reason}, index=[0])

            df = pd.concat([df, new_row]).reset_index(drop = True)
            
            winner, reason = run_simulation("random_player", f"MC_tests.team41_A3_MC_{setup[0]}_{setup[1]}_{setup[2]}", f'boards//{board}', 0.5)
            result = {0: 0, 1:-1, 2:1}[winner]
            result = {0: 0, 1:1, 2:-1}[winner]
            new_row = pd.DataFrame({"board": board[:-4], "starting": True, "C": setup[0], "totals": setup[1],
                                    "selection": setup[2], "result": result, "reason": reason}, index=[0])

            df = pd.concat([df, new_row]).reset_index(drop = True)
        
        df.to_csv(f'MC_tests//results.csv', index = False)

  0%|                                                                                           | 0/16 [00:00<?, ?it/s]

easy-2x2.txt
easy-3x3.txt
empty-2x2.txt
empty-2x3.txt
empty-3x3.txt


In [ ]:
def play_game(setup):
    if setup[2]:
        winner, reason = run_simulation("team41_A2", "team41_A1", f'boards//{setup[0]}', setup[1])
        result = {0: 0, 1:1, 2:-1}[winner]

    else:
        winner, reason = run_simulation("team41_A1", "team41_A2", f'boards//{setup[0]}', setup[1])
        result = {0: 0, 1:-1, 2:1}[winner]

In [ ]:
opponents = ["random_player"]
boards = os.listdir("boards")
times = [0.5]
starting = [True, False]

In [ ]:
testing = "team41_A2"
df = pd.DataFrame(columns = ["opponent", "board", "time", "starting", "result", "reason"])
counter = 0

for setup in tqdm(list(itertools.product(boards, times, starting))):
    if setup[2]:
        winner, reason = run_simulation("team41_A2", "team41_A1", f'boards//{setup[0]}', setup[1])
        result = {0: 0, 1:1, 2:-1}[winner]

    else:
        winner, reason = run_simulation("team41_A1", "team41_A2", f'boards//{setup[0]}', setup[1])
        result = {0: 0, 1:-1, 2:1}[winner]

    new_row = pd.DataFrame({"opponent":"team41_A1", "board": setup[0][:-4], "time": setup[1], 
                            "starting": setup[2], "result": result, "reason": reason}, index=[0])

    df = pd.concat([df, new_row]).reset_index(drop = True)
    
    counter += 1
    if counter % 10 == 0:
        df.to_csv(f'testing results//A2vA1.csv', index = False)

df.to_csv(f'testing results//A2vA1.csv', index = False)